In [16]:
import geopandas as gpd
import sqlite3

import numpy as np
import pandas as pd
from shapely import wkt
import pyogrio
from tqdm import tqdm

data_raw = "C:/Users/eleonore.kong/jll.spear/Nicolas Le Corvec - RGA/harmonic_data_tuiles_manquantes.db"
id_geom = "C:/Users/eleonore.kong/Documents/InSAR/DATA/EGMS_pid_geometry.db"
hex_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/grid.gpkg"
geopackage_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/harmonic_egms_data_v2.gpkg"
grids = ['grid_1km', 'grid_500m']

In [17]:
# Connect to the SQLite database
conn = sqlite3.connect(data_raw)
# conn_pid = sqlite3.connect(id_geom)

query = "SELECT name FROM sqlite_master WHERE type='table';"
query_pid = "SELECT * FROM points"

# Execute the query and get the table names
tables = pd.read_sql_query(query, conn)
# geom = pd.read_sql_query(query_pid, conn_pid)
# conn_pid.close()

# hex = [gpd.read_file(hex_path, layer=layer, columns=['id', 'geom']) for layer in grids]
# hex = dict(zip(grids, hex))

print(tables['name'].values)

['harmonic_output_EGMS_L3_E37N26_100km_U'
 'harmonic_output_EGMS_L3_E41N27_100km_U']


In [13]:
# def med_seasonal_amplitude(row, season):
#     years = [2016,2017,2018,2019,2020,2021]
#     if season == 'winter':
#         years = [year-1 for year in years]
#     columns = ['Detrended Amplitude_' + season + '_' + str(year) for year in years]
#     amp = [row[column] for column in columns]
#     med = np.median(amp)
#     return med

In [18]:
# Query the table
print('Found {} tables'.format(len(tables['name'].values)))
i=1

for table_name in tqdm(tables['name'].values):

    query = (
            "SELECT * FROM " + table_name +
            " WHERE \"Detrended RMS Error\" < 10"
    )

    # Read the data into a Pandas DataFrame
    df = pd.read_sql_query(query, conn)
    # df.drop('geometry', axis=1, inplace=True)
    # df = df[(df['Phase Angle (theta)'] < -1.5) | (df['Phase Angle (theta)'] > 1.5)]
    # df = df[(df['Detrended RMS Error'] < 20)]
    # columns_to_keep = [column for column in df.columns if 'Detrended' in column]
    # max_columns = [column for column in df.columns if 'Max' in column]
    # min_columns = [column for column in df.columns if 'Min' in column]
    # mean_columns = [column for column in df.columns if 'Mean' in column]
    # columns_to_keep.extend(max_columns)
    # columns_to_keep.extend(min_columns)
    # columns_to_keep.extend(mean_columns)
    columns_to_keep = [column for column in df.columns if not 'Score' in column]
    columns_to_keep = [column for column in columns_to_keep if not 'Phase' in column]
    # columns_to_keep.append('Phase Angle (theta)')
    # columns_to_keep.extend(['pid', 'geometry'])
    df = df[columns_to_keep].copy()
    df['First_peak_month'] = df['Detrended First Peak Datetime'].str.split('-', expand=True).iloc[:,1].astype(int)
    df['First_trough_month'] = df['Detrended First Trough Datetime'].str.split('-', expand=True).iloc[:,1].astype(int)
    df['peak_trough_gap'] = abs(df['First_trough_month'] - df['First_peak_month'])
    
    df = df[(df['peak_trough_gap'] >= 5) & (df['peak_trough_gap'] <= 7) & (df['First_peak_month'] < 4)]
    
  

    # for year in [2016,2017,2018,2019,2020,2021]:
    #     df['Detrended Amplitude_'+str(year)] = np.mean([df['Detrended Amplitude_winter_'+str(year-1)], df['Detrended Amplitude_summer_'+str(year)]], axis=0)
    # df['Detrended Amplitude winter'] = df.apply(lambda row: med_seasonal_amplitude(row, 'winter'), axis=1)
    # df['Detrended Amplitude summer'] = df.apply(lambda row: med_seasonal_amplitude(row, 'summer'), axis=1)
    # df['Detrended Amplitude summer-winter diff'] = df['Detrended Amplitude summer'] - df['Detrended Amplitude winter']
    # 
    # pid_list = df['pid'].to_list()    # 
    # pid_geom = geom[geom['pid'].isin(pid_list)].copy()
    # pid_geom['geometry'] = pid_geom['geometry'].apply(wkt.loads)
    # # 
    # df_geom = df.merge(pid_geom, how='left', on='pid')
    df['geometry'] = df['geometry'].apply(wkt.loads)
    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326').to_crs(epsg='2154')
    gdf = gdf.reset_index()

    # for k, df in hex.items():
    #     join = gpd.sjoin(gdf, df, how='inner', predicate='intersects')
    #     join = dict(zip(join['index'].to_list(), join['id'].to_list()))
    #     gdf[k] = gdf['index'].map(join)

    # gdf = gdf.dropna(how='all', subset=grids)
    if i == 0:
        pyogrio.write_dataframe(gdf, geopackage_path, layer="data", driver="GPKG")
    else:
        pyogrio.write_dataframe(gdf, geopackage_path, layer="data", driver="GPKG", append=True)
        
    i+=1
    
# Close the connection
conn.close()

Found 2 tables


100%|██████████| 2/2 [01:10<00:00, 35.09s/it]
